# 7.18 Homework Assignment
By Rob Finch

For this assignment, we were asked to choose a free open dataset from Kaggle and make some analyses using logistic regression and methods of classification.  I chose a dataset from Kaggle that discusses 120 years worth of Olympic athelete data.  The data includes name, age, gender, country of origin, sport and which Olympic Games(') they participated in.

# Classification Assignment

The main goal of this assignment is to check in on your ability to access, load, explore, and make predictions using classification models.  For next Wednesday, you are to do the following:

1. Locate a dataset on Kaggle, NYC Open Data, UCI Machine Learning Repository, or other resource that contains data that can be addressed through a classification task
2. Load and explore the data for missing values and perform a brief EDA
3. Frame and state the classification problem
4. Split your data into train and test sets
4. Implement a `DummyClassifier`, `KNeighborsClassifier`, and `LogisticRegression` model.
5. Improve the models by performing a `GridSearchCV` for `n_neighbors` and `C` parameters respectively.  Include a scale transformation in your pipeline for KNN and a `PolynomialFeatures` step in the Logistic model.
6. Discuss the outcome of your classifiers using the `classification_report`.  Which did the best?  Do you prefer a recall or a precision oriented model?  Why?

**EXTRA**:

- Include `SGDClassifier`
- Incorporate AUC and ROC curves


In [1]:
import sklearn
import pandas as pd
import numpy as np

In [2]:
athletes = pd.read_csv('data/athlete_events.csv')
athletes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
ID        271116 non-null int64
Name      271116 non-null object
Sex       271116 non-null object
Age       261642 non-null float64
Height    210945 non-null float64
Weight    208241 non-null float64
Team      271116 non-null object
NOC       271116 non-null object
Games     271116 non-null object
Year      271116 non-null int64
Season    271116 non-null object
City      271116 non-null object
Sport     271116 non-null object
Event     271116 non-null object
Medal     39783 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


The first thing I notice is the Medal column; it's so empty because most Olympians don't actually end up medalling.  So, I'm going to go ahead and change the null values to just say "No Medal"

In [3]:
athletes['Medal'] = athletes['Medal'].fillna('No Medal')

I have plenty of data to work with, and with a relatively small number of missing items, I feel comfortable dropping from the dataframe any line item that's missing an age, height or weight.

In [4]:
athletes = athletes.dropna(how='any')
athletes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206165 entries, 0 to 271115
Data columns (total 15 columns):
ID        206165 non-null int64
Name      206165 non-null object
Sex       206165 non-null object
Age       206165 non-null float64
Height    206165 non-null float64
Weight    206165 non-null float64
Team      206165 non-null object
NOC       206165 non-null object
Games     206165 non-null object
Year      206165 non-null int64
Season    206165 non-null object
City      206165 non-null object
Sport     206165 non-null object
Event     206165 non-null object
Medal     206165 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 25.2+ MB


In [5]:
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",No Medal
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,No Medal


I also need to transform some of the series that I want to analyze into numerical values:

In [6]:
athletes['Medal'] = athletes['Medal'].map({"No Medal" : 0, "Bronze" : 1, "Silver": 1, "Gold" : 3})
athletes['Sex'] = athletes['Sex'].map({"M" : 0, "F" : 1})
athletes['Season'] = athletes['Season'].map({"Summer" : 0, "Winter" : 1})

Now that I have a cleaned-up dataset to work with, we can start to set up our analysis.

In [7]:
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,0,24.0,180.0,80.0,China,CHN,1992 Summer,1992,0,Barcelona,Basketball,Basketball Men's Basketball,0
1,2,A Lamusi,0,23.0,170.0,60.0,China,CHN,2012 Summer,2012,0,London,Judo,Judo Men's Extra-Lightweight,0
4,5,Christine Jacoba Aaftink,1,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,1,Calgary,Speed Skating,Speed Skating Women's 500 metres,0
5,5,Christine Jacoba Aaftink,1,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,1,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",0
6,5,Christine Jacoba Aaftink,1,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,1,Albertville,Speed Skating,Speed Skating Women's 500 metres,0


### Our analysis problem is - let's try to model what makes it more likely for you to be a medal winner.  First we'll start with logistic regression.

In [8]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
corr = athletes.corr()

In [10]:
corr.Medal

ID        0.014778
Sex       0.013132
Age       0.021518
Height    0.080854
Weight    0.079300
Year     -0.037554
Season   -0.028063
Medal     1.000000
Name: Medal, dtype: float64

In [11]:
X = athletes[['Height', 'Weight']]
y = athletes[['Medal']]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
pred = lr.predict(X_test)
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.85      1.00      0.92     43977
          1       0.00      0.00      0.00      5063
          3       0.00      0.00      0.00      2502

avg / total       0.73      0.85      0.79     51542



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
lr.score(X_test, y_test)

0.8532264948973652

### Now, let's investigate a K-nearest neighbors analysis.

In [16]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
regress = KNeighborsClassifier(n_neighbors=4)

In [17]:
regress.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')

In [18]:
pred = regress.predict(X_test)

In [19]:
print(classification_report(pred, y_test))

             precision    recall  f1-score   support

          0       0.99      0.85      0.92     50937
          1       0.01      0.15      0.02       429
          3       0.01      0.10      0.01       176

avg / total       0.98      0.85      0.91     51542



### and finally, DummyClassifier.

In [21]:
from sklearn.dummy import DummyClassifier
dum = DummyClassifier(strategy='most_frequent')

In [22]:
dum.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [48]:
dum.predict(X_test)

AttributeError: 'list' object has no attribute 'argmax'

So far, it looks like K-nearest neighbors does the best job of even coming close to predicting medal winners based upon height and weight.

In [24]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

In [25]:
params_knn = {'n_neighbors': [i for i in range(3, 14)]}

In [27]:
grid = GridSearchCV(KNeighborsRegressor(), param_grid=params_knn, scoring = 'mean_squared_error')

In [28]:
grid.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/m

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda3/lib/python3.6/site-packages/sklearn/m

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='mean_squared_error', verbose=0)

In [29]:
grid.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=13, p=2,
          weights='uniform')

In [30]:
regress_best = KNeighborsClassifier(n_neighbors=13)

In [31]:
regress_best.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=13, p=2,
           weights='uniform')

In [32]:
pred_best = regress_best.predict(X_test)

In [33]:
print(classification_report(pred_best, y_test))

             precision    recall  f1-score   support

          0       1.00      0.85      0.92     51501
          1       0.00      0.26      0.00        19
          3       0.00      0.09      0.00        22

avg / total       1.00      0.85      0.92     51542



We have some movement on our K-nearest neighbors exercise through using the gridsearch function to find our ideal number of neighbors, but it is still not that significant.

In [34]:
pipe = make_pipeline(PolynomialFeatures(), LogisticRegression())

In [35]:
params_lr = {'polynomialfeatures__degree': [i for i in range (1,10)],
         'logisticregression__C': [0.1, 1, 5, 10, 100]}

In [36]:
grid_lr = GridSearchCV(pipe, param_grid=params_lr)

In [37]:
grid_lr.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'polynomialfeatures__degree': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'logisticregression__C': [0.1, 1, 5, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
best = grid.best_estimator_

In [39]:
pred = best.predict(X_test)

In [47]:
print(classification_report(pred, y_test))

ValueError: Mix type of y not allowed, got types {'multiclass', 'continuous'}

## Outcomes

Ultimately, it turns out that this was generally not a great Kaggle dataset to leverage when looking at qualitative data and classifying models.  There is a good lesson to be learned here about data selection moving forward.

When examing the actual math we did, it seems that k-nearest neighbors was the best method for predicting whether an Olympian would medal against their height and weight.  This was further enhanced by leveraging GridSearch to find the optimal number of nearest neighbors to use.  I would recommend using a precision score as the main KPI for this model/analysis; recall would include too many false positives, and when looking at the dataset we'd rather get fewer right than more wrong.